<a href="https://colab.research.google.com/github/jpoist97/lab-6-jpoist97/blob/main/DATA_301_Lab_6A_Justin_Poist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CityBikes Rest API

This section asks you to fetch JSON data from the [CityBikes REST API](https://api.citybik.es/v2/) to answer some questions about bike share programs.

Please be mindful not to send requests to the server more often than necessary. In particular, make sure you separate code that requests data from the server into a separate cell so that you do not have to repeatedly request data from the server. You will lose points if your requests are not separated into their own cell.

In [ ]:
import requests
response = requests.get("http://api.citybik.es/v2/networks")
city_bikes_json = response.json()

# Question 1

Find all cities with bike share programs in the United States (country code "US"), along with their network ID (for example, "ford-gobike"). How many cities in the U.S. have bike share programs (at least that are in this API)?

In [ ]:
import pandas as pd

df_city_bikes = pd.json_normalize(city_bikes_json, "networks")
df_city_bikes_us = df_city_bikes[df_city_bikes["location.country"] == "US"]
df_city_bikes_us[["id", "location.city"]]

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url


In [ ]:
len(df_city_bikes_us["location.city"].unique())

64

There are 64 US cities that have bike share programs.

# Question 2

Construct a `DataFrame` containing data about all bike stations in all networks in the United States. Save this `DataFrame` to disk using `.to_csv()`. (You will need it in Part B of this lab.)

Use this `DataFrame` to determine the total number of bicycles in bike share programs across the United States? You may assume that the number of bikes at a station is the number of empty spaces, plus the number of available bikes.

In [ ]:
import time
series_network_id = df_city_bikes_us["id"]

bike_stations = []

for network_id in series_network_id:
  network_response = requests.get("http://api.citybik.es/v2/networks/" + network_id)
  network = network_response.json()["network"]

  for station in network["stations"]:
    station["network_id"] = network_id
    bike_stations.append(station)
  
  time.sleep(0.3)

In [ ]:
df_bike_stations = pd.json_normalize(bike_stations)
df_bike_stations.to_csv('bike_stations.csv', index=False, header=True)

In [ ]:
df_bike_stations["empty_slots"].sum() + df_bike_stations["free_bikes"].sum() 

130660.0

The total number of bicycles in bike share programs across the United States is 130431.

# Question 3

You have just finished touring Coit Tower in San Francisco, which is located at latitude 37.802747 and longitude -122.405861. Using your `DataFrame` from Question 2, find the nearest bike station with an available bike, based on taxicab distance (a.k.a. Manhattan distance).

_Hint:_ You can check your answer using Google Maps!

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances

current_point = [37.802747, -122.405861]

def map_distances(row):
  distances = manhattan_distances([current_point], [[row["location.latitude"], row["location.longitude"]]])
  return distances[0][0]

df_city_bikes_us["distance"] = df_city_bikes_us[["location.latitude", "location.longitude"]].apply(map_distances, axis=1)
min_idx = df_city_bikes_us["distance"].idxmin()
df_city_bikes_us.loc[min_idx]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


company                                          [Motivate LLC]
href                                    /v2/networks/bay-wheels
id                                                   bay-wheels
name                                                 Bay Wheels
location.city                        San Francisco Bay Area, CA
location.country                                             US
location.latitude                                     37.714145
location.longitude                                      -122.25
source                                                      NaN
gbfs_href             https://gbfs.baywheels.com/gbfs/gbfs.json
license.name                                                NaN
license.url                                                 NaN
distance                                               0.244463
Name: 645, dtype: object

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6A - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.